In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patheffects as path_effects
from matplotlib.patches import Rectangle, PathPatch
from matplotlib.path import Path
import pandas as pd
from scipy.interpolate import make_interp_spline

def create_cold_pool_visualization(data_file='uinta_cross_section.csv',
                                 output_file='uinta_cold_pool_poster.png',
                                 dpi=300,
                                 cold_pool_height=300):  # Height of cold pool in meters
    """
    Create a professional visualization of the Uinta Basin cold pool
    for a scientific poster.

    Args:
        data_file (str): Path to CSV with cross-section data
        output_file (str): Path to save the output figure
        dpi (int): Resolution of output figure
        cold_pool_height (float): Approximate height of cold pool in meters
    """
    # Set style for professional plotting
    plt.style.use('seaborn-v0_8-whitegrid')

    # Read the data
    data = pd.read_csv(data_file)
    distances = data['distance_km']
    elevations = data['elevation_m']

    # Create smooth lines using spline interpolation
    X_smooth = np.linspace(distances.min(), distances.max(), 500)
    spl = make_interp_spline(distances, elevations, k=3)
    Y_smooth = spl(X_smooth)

    # Create the figure with a specific size for poster
    fig, ax = plt.subplots(figsize=(15, 8))

    # Plot the terrain profile
    terrain_line = ax.plot(X_smooth, Y_smooth, 'k-', linewidth=2, zorder=3,
                          label='Terrain Profile')

    # Fill terrain
    ax.fill_between(X_smooth, Y_smooth, np.min(Y_smooth),
                    color='lightgray', alpha=0.3, zorder=2)

    # Create cold pool visualization
    # Get the minimum elevation for the cold pool base
    min_elev = np.min(Y_smooth)

    # Create a mask for the cold pool area
    cold_pool_top = min_elev + cold_pool_height
    cold_pool_mask = Y_smooth <= cold_pool_top

    # Plot cold pool area
    ax.fill_between(X_smooth[cold_pool_mask],
                    Y_smooth[cold_pool_mask],
                    np.ones_like(X_smooth[cold_pool_mask]) * min_elev,
                    color='lightblue', alpha=0.4, zorder=4,
                    label='Cold Air Pool')

    # Add cold pool boundary line
    ax.plot(X_smooth[cold_pool_mask],
            np.minimum(Y_smooth[cold_pool_mask],
                      np.ones_like(X_smooth[cold_pool_mask]) * cold_pool_top),
            'b--', linewidth=1.5, alpha=0.6, zorder=5)

    # Customize the plot
    ax.set_xlabel('Distance (km)', fontsize=12, fontweight='bold')
    ax.set_ylabel('Elevation (m)', fontsize=12, fontweight='bold')
    ax.set_title('Uinta Basin Cross-Section with Cold Air Pool',
                fontsize=14, fontweight='bold', pad=20)

    # Add legend
    ax.legend(loc='upper right', frameon=True, fontsize=10)

    # Add annotations
    # Calculate positions for annotations
    mid_x = np.mean(X_smooth[cold_pool_mask])
    mid_y = min_elev + cold_pool_height/2

    # Add cold pool annotation
    text = ax.text(mid_x, mid_y, 'Cold Air Pool\nInversion Layer',
                  ha='center', va='center', fontsize=10,
                  color='darkblue', fontweight='bold')
    text.set_path_effects([path_effects.withStroke(linewidth=2,
                                                 foreground='white')])

    # Add elevation markers
    ax.axhline(y=min_elev, color='gray', linestyle=':', alpha=0.5)
    ax.axhline(y=cold_pool_top, color='gray', linestyle=':', alpha=0.5)

    # Add elevation labels
    ax.text(ax.get_xlim()[1], min_elev,
            f'{int(min_elev)}m', va='bottom', ha='right')
    ax.text(ax.get_xlim()[1], cold_pool_top,
            f'{int(cold_pool_top)}m', va='bottom', ha='right')

    # Add compass direction
    ax.text(ax.get_xlim()[0], ax.get_ylim()[1], 'SW',
            fontsize=10, fontweight='bold')
    ax.text(ax.get_xlim()[1], ax.get_ylim()[1], 'NE',
            fontsize=10, fontweight='bold')

    # Customize grid
    ax.grid(True, linestyle='--', alpha=0.7)

    # Add scale bar
    scale_bar_length = 10  # km
    x_min, x_max = ax.get_xlim()
    y_min, y_max = ax.get_ylim()
    scale_bar_x = x_min + (x_max - x_min) * 0.05
    scale_bar_y = y_min + (y_max - y_min) * 0.1

    ax.plot([scale_bar_x, scale_bar_x + scale_bar_length],
            [scale_bar_y, scale_bar_y], 'k-', linewidth=2)
    ax.text(scale_bar_x + scale_bar_length/2, scale_bar_y - (y_max - y_min)*0.02,
            f'{scale_bar_length} km', ha='center')

    # Add temperature gradient arrow
    arrow_x = x_min + (x_max - x_min) * 0.15
    arrow_bottom = min_elev
    arrow_top = cold_pool_top + 100

    # Create temperature gradient arrow
    ax.annotate('', xy=(arrow_x, arrow_top),
                xytext=(arrow_x, arrow_bottom),
                arrowprops=dict(arrowstyle='->',
                              color='red',
                              lw=2))

    # Add temperature labels
    ax.text(arrow_x + (x_max - x_min)*0.01, arrow_bottom, 'Cold',
            va='bottom', ha='left', color='blue')
    ax.text(arrow_x + (x_max - x_min)*0.01, arrow_top, 'Warm',
            va='top', ha='left', color='red')

    # Tight layout
    plt.tight_layout()

    # Save the figure
    plt.savefig(output_file, dpi=dpi, bbox_inches='tight')
    plt.close()

def create_3d_visualization(data_file='uinta_cross_section.csv',
                          output_file='uinta_cold_pool_3d.png',
                          dpi=300):
    """
    Create a 3D visualization of the cold pool
    """
    from mpl_toolkits.mplot3d import Axes3D

    # Read the data
    data = pd.read_csv(data_file)
    distances = data['distance_km']
    elevations = data['elevation_m']

    # Create smooth lines
    X_smooth = np.linspace(distances.min(), distances.max(), 100)
    spl = make_interp_spline(distances, elevations, k=3)
    Y_smooth = spl(X_smooth)

    # Create 3D figure
    fig = plt.figure(figsize=(15, 10))
    ax = fig.add_subplot(111, projection='3d')

    # Create mesh grid for 3D surface
    X, Z = np.meshgrid(X_smooth, np.linspace(0, 5, 50))
    Y = np.array([Y_smooth for _ in range(50)])

    # Plot the surface
    surf = ax.plot_surface(X, Z, Y, cmap='terrain',
                          linewidth=0, antialiased=True)

    # Customize the plot
    ax.set_xlabel('Distance (km)')
    ax.set_ylabel('Basin Width (km)')
    ax.set_zlabel('Elevation (m)')

    # Add colorbar
    fig.colorbar(surf, ax=ax, shrink=0.5, aspect=5)

    # Set the viewing angle
    ax.view_init(elev=20, azim=45)

    # Save the figure
    plt.savefig(output_file, dpi=dpi, bbox_inches='tight')
    plt.close()

if __name__ == '__main__':
    # Create both 2D and 3D visualizations
    create_cold_pool_visualization()
    create_3d_visualization()

    print("Visualizations have been created!")

Visualizations have been created!


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patheffects as path_effects
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.patches import FancyArrowPatch, Rectangle
import pandas as pd
from scipy.interpolate import make_interp_spline
import matplotlib.font_manager as fm

def create_conference_visualization(data_file='uinta_cross_section.csv',
                                 output_file='uinta_cold_pool_conference.png',
                                 dpi=600):
    """
    Create a high-impact conference visualization of the Uinta Basin cold pool
    designed for maximum visibility and clarity in a large venue.
    """
    # Set up the plot style for maximum impact
    plt.style.use('dark_background')

    # Create figure with precise dimensions for poster visibility
    fig = plt.figure(figsize=(20, 12))

    # Create two subplot areas - main plot and inset map
    gs = fig.add_gridspec(1, 1)
    ax = fig.add_subplot(gs[0, 0])

    # Read and smooth the data
    data = pd.read_csv(data_file)
    distances = data['distance_km']
    elevations = data['elevation_m']

    # Create smooth lines using spline interpolation
    X_smooth = np.linspace(distances.min(), distances.max(), 1000)
    spl = make_interp_spline(distances, elevations, k=3)
    Y_smooth = spl(X_smooth)

    # Calculate cold pool parameters
    min_elev = np.min(Y_smooth)
    cold_pool_height = 300  # meters
    cold_pool_top = min_elev + cold_pool_height

    # Create custom color gradients
    terrain_colors = ['#2d1606', '#654321', '#8b4513', '#a0522d']
    terrain_cmap = LinearSegmentedColormap.from_list('terrain', terrain_colors)

    cold_pool_colors = ['#001137', '#002255', '#003373']
    cold_pool_cmap = LinearSegmentedColormap.from_list('cold_pool', cold_pool_colors, N=100)

    # Plot terrain with sophisticated styling
    ax.fill_between(X_smooth, Y_smooth, min_elev - 100,
                    color='none',
                    linewidth=0,
                    zorder=2)

    # Add terrain texture
    for i in range(20):
        offset = i * 20
        alpha = 0.1 - (i * 0.005)
        ax.fill_between(X_smooth, Y_smooth + offset, min_elev - 100,
                       color='#4a3828',
                       alpha=alpha,
                       linewidth=0,
                       zorder=2)

    # Plot the cold pool with gradient effect
    cold_pool_mask = Y_smooth <= cold_pool_top

    # Create gradient effect for cold pool
    num_layers = 50
    for i in range(num_layers):
        fraction = i / num_layers
        height = min_elev + (cold_pool_height * fraction)
        alpha = 0.02
        color = cold_pool_cmap(fraction)

        ax.fill_between(X_smooth[cold_pool_mask],
                       np.minimum(Y_smooth[cold_pool_mask], height),
                       min_elev,
                       color=color,
                       alpha=alpha,
                       zorder=3)

    # Add atmospheric layers above cold pool
    layer_colors = ['#FFB6C1', '#ADD8E6', '#87CEEB']
    layer_heights = [cold_pool_top + 200, cold_pool_top + 400, cold_pool_top + 600]

    for i, (color, height) in enumerate(zip(layer_colors, layer_heights)):
        ax.fill_between(X_smooth,
                       height,
                       height - 200,
                       color=color,
                       alpha=0.1,
                       zorder=1)

    # Add temperature inversion arrow
    arrow_x = X_smooth[0] + (X_smooth[-1] - X_smooth[0]) * 0.1
    arrow_props = dict(arrowstyle='->',
                      linewidth=3,
                      color='white',
                      mutation_scale=25)

    # Temperature gradient arrow
    temperature_arrow = FancyArrowPatch((arrow_x, min_elev + 100),
                                      (arrow_x, cold_pool_top + 200),
                                      connectionstyle="arc3,rad=0",
                                      **arrow_props)
    ax.add_patch(temperature_arrow)

    # Add labels with enhanced visibility
    def add_outlined_text(x, y, text, **kwargs):
        txt = ax.text(x, y, text, **kwargs)
        txt.set_path_effects([
            path_effects.Stroke(linewidth=3, foreground='black'),
            path_effects.Normal()
        ])
        return txt

    # Add temperature labels
    add_outlined_text(arrow_x + 5, min_elev + 50, 'Cold Air Pool\n(-10°C)',
                     color='#00BFFF', fontsize=14, fontweight='bold', ha='left')
    add_outlined_text(arrow_x + 5, cold_pool_top + 250, 'Warm Air\n(5°C)',
                     color='#FF6B6B', fontsize=14, fontweight='bold', ha='left')

    # Add elevation markers
    ax.axhline(y=min_elev, color='white', linestyle=':', alpha=0.3)
    ax.axhline(y=cold_pool_top, color='white', linestyle=':', alpha=0.3)

    # Add professional labels
    elevation_fontsize = 12
    add_outlined_text(ax.get_xlim()[1], min_elev,
                     f'{int(min_elev)}m', fontsize=elevation_fontsize,
                     va='bottom', ha='right', color='white')
    add_outlined_text(ax.get_xlim()[1], cold_pool_top,
                     f'{int(cold_pool_top)}m', fontsize=elevation_fontsize,
                     va='bottom', ha='right', color='white')

    # Add title and labels
    title = ax.set_title('Uinta Basin Cold Air Pool Formation',
                        fontsize=24, pad=20, color='white', fontweight='bold')
    title.set_path_effects([path_effects.withStroke(linewidth=3,
                                                  foreground='black')])

    ax.set_xlabel('Distance (km)', fontsize=16, color='white', labelpad=10)
    ax.set_ylabel('Elevation (m)', fontsize=16, color='white', labelpad=10)

    # Customize grid
    ax.grid(True, alpha=0.2, color='white', linestyle='--')

    # Add compass directions
    add_outlined_text(X_smooth[0], ax.get_ylim()[1], 'SW',
                     fontsize=16, fontweight='bold', color='white')
    add_outlined_text(X_smooth[-1], ax.get_ylim()[1], 'NE',
                     fontsize=16, fontweight='bold', color='white')

    # Add scale bar
    scale_bar_length = 10  # km
    scale_bar_x = X_smooth[0] + (X_smooth[-1] - X_smooth[0]) * 0.05
    scale_bar_y = min_elev - 100

    ax.plot([scale_bar_x, scale_bar_x + scale_bar_length],
            [scale_bar_y, scale_bar_y],
            'white', linewidth=3)
    add_outlined_text(scale_bar_x + scale_bar_length/2,
                     scale_bar_y - 50,
                     f'{scale_bar_length} km',
                     ha='center',
                     color='white',
                     fontsize=12)

    # Add explanatory text
    explanation = (
        'Cold Air Pool Formation:\n'
        '• Dense cold air sinks to valley bottom\n'
        '• Warm air rises and caps the basin\n'
        '• Temperature inversion traps pollutants'
    )

    # Add explanation box
    ax.text(0.98, 0.02, explanation,
            transform=ax.transAxes,
            color='white',
            fontsize=14,
            ha='right',
            va='bottom',
            bbox=dict(facecolor='black',
                     alpha=0.7,
                     edgecolor='white',
                     boxstyle='round,pad=1'))

    # Adjust layout and save
    plt.tight_layout()
    plt.savefig(output_file,
                dpi=dpi,
                bbox_inches='tight',
                facecolor='black',
                edgecolor='none')
    plt.close()

if __name__ == '__main__':
    create_conference_visualization()
    print("High-impact conference visualization has been created!")

High-impact conference visualization has been created!


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patheffects as path_effects
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.patches import FancyArrowPatch, Rectangle
import pandas as pd
from scipy.interpolate import make_interp_spline

def create_conference_visualization(data_file='uinta_cross_section.csv',
                                 output_file='uinta_cold_pool_conference.png',
                                 dpi=600):
    """
    Create a high-impact conference visualization of the Uinta Basin cold pool
    with landmarks, cities, and improved temperature gradients.
    """
    plt.style.use('dark_background')
    fig = plt.figure(figsize=(24, 14))
    ax = fig.add_subplot(111)

    # Read and smooth the data
    data = pd.read_csv(data_file)
    distances = data['distance_km']
    elevations = data['elevation_m']

    X_smooth = np.linspace(distances.min(), distances.max(), 1000)
    spl = make_interp_spline(distances, elevations, k=3)
    Y_smooth = spl(X_smooth)

    # Calculate cold pool parameters
    min_elev = np.min(Y_smooth)
    cold_pool_height = 300  # meters
    cold_pool_top = min_elev + cold_pool_height

    # Create enhanced temperature gradient (dark blue to light blue)
    cold_pool_colors = ['#00008B',  # Dark blue (coldest)
                       '#000099',
                       '#0000AA',
                       '#0000BB',
                       '#0000CC',
                       '#0000DD',
                       '#0000EE',
                       '#0000FF',
                       '#1E90FF',   # Lighter blue
                       '#87CEEB']   # Light sky blue (warmest)

    cold_pool_cmap = LinearSegmentedColormap.from_list('cold_pool', cold_pool_colors, N=100)

    # Plot terrain with enhanced styling
    ax.fill_between(X_smooth, Y_smooth, min_elev - 100,
                    color='#4a3828',
                    linewidth=0,
                    zorder=2)

    # Add terrain texture
    for i in range(20):
        offset = i * 20
        alpha = 0.1 - (i * 0.005)
        ax.fill_between(X_smooth, Y_smooth + offset, min_elev - 100,
                       color='#4a3828',
                       alpha=alpha,
                       linewidth=0,
                       zorder=2)

    # Plot cold pool with enhanced gradient
    cold_pool_mask = Y_smooth <= cold_pool_top
    num_layers = 50

    for i in range(num_layers):
        fraction = i / num_layers
        height = min_elev + (cold_pool_height * fraction)
        alpha = 0.03
        color = cold_pool_cmap(fraction)

        ax.fill_between(X_smooth[cold_pool_mask],
                       np.minimum(Y_smooth[cold_pool_mask], height),
                       min_elev,
                       color=color,
                       alpha=alpha,
                       zorder=3)

    # Define landmarks and cities with their approximate positions
    landmarks = {
    'Duchesne': {'x': 20, 'y': min_elev + 50, 'elevation': 5518},
    'Roosevelt': {'x': 35, 'y': min_elev + 30, 'elevation': 5250},
    'Vernal': {'x': 65, 'y': min_elev + 40, 'elevation': 5328},
    'Split Mountain': {'x': 75, 'y': Y_smooth.max() - 200, 'elevation': 8950},
    'Asphalt Ridge': {'x': 55, 'y': min_elev + 800, 'elevation': 7000},
    'Tabby Mountain': {'x': 15, 'y': Y_smooth.max() - 100, 'elevation': 10700}
}

    def add_outlined_text(x, y, text, **kwargs):
        txt = ax.text(x, y, text, **kwargs)
        txt.set_path_effects([
            path_effects.Stroke(linewidth=3, foreground='black'),
            path_effects.Normal()
        ])
        return txt

    # Add landmarks and cities with connection lines
    for name, info in landmarks.items():
        # Add dot for location
        ax.plot(info['x'], info['y'], 'o', color='white', markersize=8, zorder=5)

        # Add name and elevation with leader line
        elevation_text = f"{name}\n({info['elevation']}ft)"

        # Adjust text position based on location
        if info['y'] > (min_elev + cold_pool_height + 200):
            # For high elevation points, put text above
            text_y = info['y'] + 100
            va = 'bottom'
        else:
            # For low elevation points, put text below
            text_y = info['y'] - 100
            va = 'top'

        add_outlined_text(info['x'], text_y, elevation_text,
                         color='white', fontsize=12, ha='center', va=va,
                         fontweight='bold')

        # Add leader line
        ax.plot([info['x'], info['x']], [info['y'], text_y],
                color='white', linestyle='--', alpha=0.5, zorder=4)

    # Add temperature scale
    temp_scale_x = X_smooth[-1] * 0.95
    temp_heights = np.linspace(min_elev, cold_pool_top + 200, 6)
    temp_values = np.linspace(-10, 5, 6)

    for i in range(len(temp_heights) - 1):
        height = temp_heights[i]
        temp = temp_values[i]
        add_outlined_text(temp_scale_x, height, f"{temp}°C",
                         color='white', fontsize=10, ha='left')

    # Add title and labels
    title = ax.set_title('Uinta Basin Cold Air Pool Formation',
                        fontsize=24, pad=20, color='white', fontweight='bold')
    title.set_path_effects([path_effects.withStroke(linewidth=3,
                                                  foreground='black')])

    ax.set_xlabel('Distance (km)', fontsize=16, color='white', labelpad=10)
    ax.set_ylabel('Elevation (m)', fontsize=16, color='white', labelpad=10)

    # Add explanatory text box
    explanation = (
        'Cold Air Pool Characteristics:\n'
        '• Temperature inversion traps cold air in basin\n'
        '• Vertical temperature gradient: -10°C to 5°C\n'
        '• Affects communities from Duchesne to Vernal\n'
        '• Common during winter months\n'
        '• Can persist for days to weeks'
    )

    # Add explanation box
    ax.text(0.98, 0.02, explanation,
            transform=ax.transAxes,
            color='white',
            fontsize=14,
            ha='right',
            va='bottom',
            bbox=dict(facecolor='black',
                     alpha=0.7,
                     edgecolor='white',
                     boxstyle='round,pad=1'))

    # Add compass directions
    add_outlined_text(X_smooth[0], ax.get_ylim()[1], 'SW',
                     fontsize=16, fontweight='bold', color='white')
    add_outlined_text(X_smooth[-1], ax.get_ylim()[1], 'NE',
                     fontsize=16, fontweight='bold', color='white')

    # Add scale bar
    scale_bar_length = 10  # km
    scale_bar_x = X_smooth[0] + (X_smooth[-1] - X_smooth[0]) * 0.05
    scale_bar_y = min_elev - 100

    ax.plot([scale_bar_x, scale_bar_x + scale_bar_length],
            [scale_bar_y, scale_bar_y],
            'white', linewidth=3)
    add_outlined_text(scale_bar_x + scale_bar_length/2,
                     scale_bar_y - 50,
                     f'{scale_bar_length} km',
                     ha='center',
                     color='white',
                     fontsize=12)

    plt.tight_layout()
    plt.savefig(output_file,
                dpi=dpi,
                bbox_inches='tight',
                facecolor='black',
                edgecolor='none')
    plt.close()

if __name__ == '__main__':
    create_conference_visualization()
    print("Enhanced conference visualization has been created!")

Enhanced conference visualization has been created!


In [6]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patheffects as path_effects
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.patches import FancyArrowPatch, Rectangle, Polygon
from matplotlib.transforms import blended_transform_factory
import pandas as pd
from scipy.interpolate import make_interp_spline
from osgeo import gdal
import numpy.ma as ma
from matplotlib.gridspec import GridSpec
import matplotlib.patches as patches

def extract_elevation_profile(dem_file, start_point, end_point, num_points=1000):
    """Extract elevation profile from DEM along a line between two points."""
    dem = gdal.Open(dem_file)
    if dem is None:
        raise ValueError("Could not open DEM file")

    geotransform = dem.GetGeoTransform()
    band = dem.GetRasterBand(1)
    dem_data = band.ReadAsArray()

    lons = np.linspace(start_point[0], end_point[0], num_points)
    lats = np.linspace(start_point[1], end_point[1], num_points)

    elevations = []
    distances = []

    total_dist = np.sqrt((end_point[0] - start_point[0])**2 +
                        (end_point[1] - start_point[1])**2) * 111.32

    for i in range(num_points):
        px = int((lons[i] - geotransform[0]) / geotransform[1])
        py = int((lats[i] - geotransform[3]) / geotransform[5])

        try:
            elevation = dem_data[py, px]
            if elevation != band.GetNoDataValue():
                elevations.append(elevation)
                distances.append(total_dist * i / num_points)
        except:
            continue

    return np.array(distances), np.array(elevations), dem_data, geotransform

def create_professional_visualization(dem_file='uinta_merged.tif',
                                   output_file='uinta_basin_presentation.png',
                                   dpi=800):
    """Create a publication-quality visualization of the Uinta Basin cold pool."""

    # Set up the figure with multiple panels
    plt.style.use('dark_background')
    fig = plt.figure(figsize=(30, 20))
    gs = GridSpec(2, 3, height_ratios=[3, 1], width_ratios=[1, 1, 0.8])

    # Main cross-section panel
    ax_main = fig.add_subplot(gs[0, :])

    # Supporting panels
    ax_map = fig.add_subplot(gs[1, 0])
    ax_temp = fig.add_subplot(gs[1, 1])
    ax_legend = fig.add_subplot(gs[1, 2])

    def add_outlined_text(ax, x, y, text, **kwargs):
        txt = ax.text(x, y, text, **kwargs)
        txt.set_path_effects([
            path_effects.Stroke(linewidth=3, foreground='black'),
            path_effects.Normal()
        ])
        return txt

    # Extract elevation data
    start_point = (-110.1, 40.0)  # SW point
    end_point = (-109.2, 40.5)    # NE point

    try:
        # Get elevation data from DEM
        distances, elevations, dem_data, geotransform = extract_elevation_profile(
            dem_file, start_point, end_point
        )

        # Create smooth elevation profile
        X_smooth = np.linspace(distances.min(), distances.max(), 1000)
        spl = make_interp_spline(distances, elevations, k=3)
        Y_smooth = spl(X_smooth)

        # Calculate cold pool parameters
        min_elev = np.min(Y_smooth)
        cold_pool_height = 300  # meters
        cold_pool_top = min_elev + cold_pool_height

        # Plot terrain with enhanced styling
        ax_main.fill_between(X_smooth, Y_smooth, min_elev - 100,
                           color='#4a3828',
                           linewidth=0,
                           zorder=2)

        # Add terrain texture
        for i in range(20):
            offset = i * 20
            alpha = 0.1 - (i * 0.005)
            ax_main.fill_between(X_smooth, Y_smooth + offset, min_elev - 100,
                               color='#4a3828',
                               alpha=alpha,
                               linewidth=0,
                               zorder=2)

        # Create enhanced cold pool color gradient
        cold_pool_colors = [
            '#000033',  # Darkest blue (coldest)
            '#000066',
            '#000099',
            '#0000CC',
            '#0000FF',
            '#4169E1',
            '#1E90FF',
            '#87CEEB',   # Light sky blue
            '#B0E0E6',   # Powder blue
            '#F0F8FF'    # Alice blue (warmest)
        ]

        cold_pool_cmap = LinearSegmentedColormap.from_list('cold_pool', cold_pool_colors, N=100)

        # Plot cold pool with gradient
        cold_pool_mask = Y_smooth <= cold_pool_top
        num_layers = 50

        for i in range(num_layers):
            fraction = i / num_layers
            height = min_elev + (cold_pool_height * fraction)
            alpha = 0.03
            color = cold_pool_cmap(fraction)

            ax_main.fill_between(X_smooth[cold_pool_mask],
                               np.minimum(Y_smooth[cold_pool_mask], height),
                               min_elev,
                               color=color,
                               alpha=alpha,
                               zorder=3)

        # Add atmospheric layers
        X_atm = np.linspace(X_smooth[0], X_smooth[-1], 1000)
        num_atm_layers = 100

        for i in range(num_atm_layers):
            y_base = np.linspace(min_elev, cold_pool_top + 500, len(X_atm))
            y_top = y_base + i*10
            y_bottom = y_base + (i+1)*10

            alpha = 0.01 * np.exp(-i/20)
            ax_main.fill_between(X_atm, y_bottom, y_top,
                               color='white', alpha=alpha, zorder=1)

        # Add wind arrows showing circulation
        wind_arrows = []
        for i in range(10):
            x = X_smooth[0] + i * (X_smooth[-1] - X_smooth[0])/10
            y = min_elev + cold_pool_height + 200
            arrow = FancyArrowPatch((x, y), (x, y-100),
                                  arrowstyle='->',
                                  color='white',
                                  alpha=0.3,
                                  mutation_scale=15)
            ax_main.add_patch(arrow)
            wind_arrows.append(arrow)

        # Define landmarks with actual elevations
        landmarks = {
            'Duchesne': {'x': 20, 'y': min_elev + 50, 'elevation': 5518},
            'Roosevelt': {'x': 35, 'y': min_elev + 30, 'elevation': 5250},
            'Vernal': {'x': 65, 'y': min_elev + 40, 'elevation': 5328},
            'Split Mountain': {'x': 75, 'y': Y_smooth.max() - 200, 'elevation': 8950},
            'Asphalt Ridge': {'x': 55, 'y': min_elev + 800, 'elevation': 7000},
            'Tabby Mountain': {'x': 15, 'y': Y_smooth.max() - 100, 'elevation': 10700}
        }

        # Add landmarks
        for name, info in landmarks.items():
            ax_main.plot(info['x'], info['y'], 'o', color='white', markersize=8, zorder=5)
            elevation_text = f"{name}\n({info['elevation']}ft)"

            if info['y'] > (min_elev + cold_pool_height + 200):
                text_y = info['y'] + 100
                va = 'bottom'
            else:
                text_y = info['y'] - 100
                va = 'top'

            add_outlined_text(ax_main, info['x'], text_y, elevation_text,
                           color='white', fontsize=12, ha='center', va=va,
                           fontweight='bold')

            ax_main.plot([info['x'], info['x']], [info['y'], text_y],
                       color='white', linestyle='--', alpha=0.5, zorder=4)

        # Add temperature profile in side panel
        heights = np.linspace(min_elev, cold_pool_top + 500, 100)
        temps = -10 + (15 * (heights - min_elev)/(cold_pool_top + 500 - min_elev))
        ax_temp.plot(temps, heights, 'r-', linewidth=2)
        ax_temp.set_title('Temperature Profile', color='white', fontsize=14)
        ax_temp.set_xlabel('Temperature (°C)', color='white')
        ax_temp.set_ylabel('Elevation (m)', color='white')
        ax_temp.grid(True, alpha=0.3)

        # Add legend and information
        legend_text = (
            'Uinta Basin Cold Pool Formation\n\n'
            'Characteristics:\n'
            '• Intense temperature inversion\n'
            '• Duration: Days to weeks\n'
            '• Vertical extent: ~300m\n'
            '• Typical ΔT: 15°C\n\n'
            'Effects:\n'
            '• Air quality degradation\n'
            '• Reduced visibility\n'
            '• Public health impacts\n'
            '• Transportation hazards'
        )
        ax_legend.text(0.05, 0.95, legend_text,
                     transform=ax_legend.transAxes,
                     color='white',
                     fontsize=12,
                     va='top')
        ax_legend.axis('off')

        # Add title to main panel
        title = ax_main.set_title('Uinta Basin Cold Air Pool',
                                fontsize=24, pad=20, color='white', fontweight='bold')
        title.set_path_effects([path_effects.withStroke(linewidth=3,
                                                     foreground='black')])

        # Add labels to main panel
        ax_main.set_xlabel('Distance (km)', fontsize=16, color='white', labelpad=10)
        ax_main.set_ylabel('Elevation (m)', fontsize=16, color='white', labelpad=10)

        # Add compass directions
        add_outlined_text(ax_main, X_smooth[0], ax_main.get_ylim()[1], 'SW',
                       fontsize=16, fontweight='bold', color='white')
        add_outlined_text(ax_main, X_smooth[-1], ax_main.get_ylim()[1], 'NE',
                       fontsize=16, fontweight='bold', color='white')

        # Add scale bar
        scale_bar_length = 10
        scale_bar_x = X_smooth[0] + (X_smooth[-1] - X_smooth[0]) * 0.05
        scale_bar_y = min_elev - 100

        ax_main.plot([scale_bar_x, scale_bar_x + scale_bar_length],
                   [scale_bar_y, scale_bar_y],
                   'white', linewidth=3)
        add_outlined_text(ax_main, scale_bar_x + scale_bar_length/2,
                       scale_bar_y - 50,
                       f'{scale_bar_length} km',
                       ha='center',
                       color='white',
                       fontsize=12)

        # Add grid to main panel
        ax_main.grid(True, alpha=0.3, linestyle='--')

        # Save the figure
        plt.savefig(output_file,
                   dpi=dpi,
                   bbox_inches='tight',
                   facecolor='black',
                   edgecolor='none')
        plt.close()

        print("Professional conference visualization created successfully!")

    except Exception as e:
        print(f"Error creating visualization: {str(e)}")
        raise

if __name__ == '__main__':
    create_professional_visualization()

Professional conference visualization created successfully!


In [1]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from osgeo import gdal

# First let's examine the TIF data
def examine_dem(dem_file='uinta_merged.tif'):
    """
    Examine the DEM data to understand what we're working with
    """
    # Open the dataset
    dem = gdal.Open(dem_file)
    if dem is None:
        print("Could not open DEM file")
        return

    # Get basic info
    print("DEM Info:")
    print(f"Driver: {dem.GetDriver().ShortName}")
    print(f"Size: {dem.RasterXSize}x{dem.RasterYSize}")
    print(f"Number of bands: {dem.RasterCount}")

    # Get geotransform
    geotransform = dem.GetGeoTransform()
    print("\nGeotransform:")
    print(f"Origin: ({geotransform[0]}, {geotransform[3]})")
    print(f"Pixel Size: ({geotransform[1]}, {geotransform[5]})")

    # Get band data
    band = dem.GetRasterBand(1)
    print("\nBand Info:")
    print(f"No Data Value: {band.GetNoDataValue()}")
    print(f"Min: {band.GetMinimum()}")
    print(f"Max: {band.GetMaximum()}")

    # Get data
    data = band.ReadAsArray()
    print("\nData Shape:", data.shape)
    print("Data Statistics:")
    print(f"Mean elevation: {np.mean(data[data != band.GetNoDataValue()]):.2f}")
    print(f"Min elevation: {np.min(data[data != band.GetNoDataValue()]):.2f}")
    print(f"Max elevation: {np.max(data[data != band.GetNoDataValue()]):.2f}")

    return dem, data, geotransform

if __name__ == '__main__':
    dem, data, geotransform = examine_dem()

DEM Info:
Driver: GTiff
Size: 13500x10800
Number of bands: 1

Geotransform:
Origin: (-110.25, 40.75)
Pixel Size: (9.259259259259259e-05, -9.259259259259259e-05)

Band Info:
No Data Value: -999999.0
Min: None
Max: None

Data Shape: (10800, 13500)
Data Statistics:
Mean elevation: 1871.72
Min elevation: 1400.05
Max elevation: 3788.14


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap, LightSource
import matplotlib.patheffects as path_effects
from matplotlib.gridspec import GridSpec
from osgeo import gdal
import matplotlib.patches as patches

def create_enhanced_visualization(dem_file='uinta_merged.tif',
                                output_file='uinta_basin_analysis.png',
                                dpi=600):
    """
    Create a comprehensive visualization of the Uinta Basin using actual DEM data
    """
    # Set up the figure
    plt.style.use('dark_background')
    fig = plt.figure(figsize=(24, 16))
    gs = GridSpec(2, 2, height_ratios=[2, 1], width_ratios=[2, 1])

    # Main topographic map
    ax_main = fig.add_subplot(gs[0, :])
    # Cross section
    ax_cross = fig.add_subplot(gs[1, 0])
    # Legend/Info panel
    ax_info = fig.add_subplot(gs[1, 1])

    # Load DEM data
    dem = gdal.Open(dem_file)
    band = dem.GetRasterBand(1)
    data = band.ReadAsArray()
    geotransform = dem.GetGeoTransform()

    # Create coordinate arrays
    ny, nx = data.shape
    x = np.linspace(geotransform[0], geotransform[0] + nx*geotransform[1], nx)
    y = np.linspace(geotransform[3], geotransform[3] + ny*geotransform[5], ny)
    X, Y = np.meshgrid(x, y)

    # Create custom colormap for elevation
    elevation_colors = ['#2d1606', '#654321', '#8b4513', '#a0522d', '#deb887']
    terrain_cmap = LinearSegmentedColormap.from_list('terrain', elevation_colors)

    # Calculate hillshade
    ls = LightSource(azdeg=315, altdeg=45)
    hillshade = ls.hillshade(data, vert_exag=3)

    # Plot main topographic map with hillshade
    rgb = ls.shade(data, cmap=terrain_cmap, blend_mode='overlay')
    ax_main.imshow(rgb, extent=[x.min(), x.max(), y.min(), y.max()])

    # Add cold pool region (simplified for visualization)
    cold_pool_mask = data < 1800  # Adjust threshold as needed
    cold_pool_overlay = np.zeros_like(data)
    cold_pool_overlay[cold_pool_mask] = 1

    # Plot cold pool with blue overlay
    ax_main.imshow(cold_pool_overlay, extent=[x.min(), x.max(), y.min(), y.max()],
                   cmap='Blues', alpha=0.3)

    # Extract and plot cross-section
    mid_row = ny // 2
    profile = data[mid_row, :]
    distance = np.linspace(0, (nx * geotransform[1]) * 111, nx)  # Convert to km

    ax_cross.plot(distance, profile, 'white', linewidth=2)
    ax_cross.fill_between(distance, profile, profile.min(),
                         color='#4a3828', alpha=0.3)

    # Add cold pool indication in cross-section
    cold_pool_height = 300  # meters
    cold_pool_base = profile.min() + cold_pool_height
    cold_pool_mask = profile < cold_pool_base
    ax_cross.fill_between(distance[cold_pool_mask],
                         profile[cold_pool_mask],
                         np.ones_like(profile[cold_pool_mask]) * cold_pool_base,
                         color='blue', alpha=0.2)

    # Add key locations
    locations = {
        'Duchesne': (-110.0, 40.2),
        'Roosevelt': (-109.9, 40.3),
        'Vernal': (-109.5, 40.4),
        'Wasatch Range': (-110.2, 40.5)
    }

    for name, (lon, lat) in locations.items():
        ax_main.plot(lon, lat, 'wo', markersize=8)
        ax_main.text(lon, lat + 0.02, name, color='white',
                    ha='center', va='bottom', fontsize=10,
                    path_effects=[path_effects.withStroke(linewidth=2,
                                                        foreground='black')])

    # Add snow shadow effect indication
    shadow_arrow = patches.FancyArrowPatch(
        (-110.2, 40.4), (-109.8, 40.4),
        connectionstyle="arc3,rad=.2",
        arrowstyle="->", color='white', alpha=0.6)
    ax_main.add_patch(shadow_arrow)
    ax_main.text(-110.0, 40.45, 'Snow Shadow\nEffect',
                 color='white', ha='center', va='bottom')

    # Add informative text
    info_text = (
        'Uinta Basin Cold Pool Analysis\n\n'
        'Key Features:\n'
        '• Snow shadow in lee of Wasatch Mountains\n'
        '• Cold pool formation in basin\n'
        '• Elevation range: 1400-3788m\n'
        '• Primary impact zone: <1800m\n\n'
        'Environmental Impact:\n'
        '• Ozone accumulation in cold pool\n'
        '• Reduced precipitation in shadow\n'
        '• Limited vertical mixing\n'
        '• Air quality concerns'
    )
    ax_info.text(0.05, 0.95, info_text, transform=ax_info.transAxes,
                 color='white', fontsize=12, va='top')
    ax_info.axis('off')

    # Add titles and labels
    ax_main.set_title('Uinta Basin Topography and Cold Pool Formation',
                      fontsize=16, pad=20)
    ax_cross.set_xlabel('Distance (km)')
    ax_cross.set_ylabel('Elevation (m)')

    # Add compass rose
    ax_main.text(x.max()-0.1, y.max()-0.1, 'N', fontsize=12,
                 ha='center', va='center', color='white')

    # Save figure
    plt.tight_layout()
    plt.savefig(output_file, dpi=dpi, bbox_inches='tight',
                facecolor='black', edgecolor='none')
    plt.close()

if __name__ == '__main__':
    create_enhanced_visualization()